### Python Assignment 1

In this assignment you will get familiar with analyzing drifter data. The drifters that we will use were released during the Australasian Antarctic Expedition in December 2013, a project designed to characterize eddy dispersion and diffusivity along an Antarctic Circumpolar Current front. For more information you may take a look at [this scientific article](https://doi.org/10.1002/2015JC010972). An animation of the drifters can be found on http://oceanparcels.org/aaemap.

We assume that you have completed all steps in the *Getting python-ready for Dynamical Oceanography* document and have a new environment called *dyoc*. Now check that you are in the right environment by running the following cell:

In [ ]:
!conda env list

You should see an asterisk next to your *dyoc* environment. If this is not the case, you may stop the *Jupyter lab* instance using `ctrl + c` in your terminal (Mac) or Anaconda prompt (Windows). Then type `conda activate dyoc` to activate the environment and `jupyter lab` to start *Jupyter lab* again.  

Now import the following packages:

In [ ]:
import numpy as np
import json
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.colors as mcolors
import matplotlib.cm
import cartopy
import cmocean
import geopy.distance

Then download the file `aaedrifters.json` from http://oceanparcels.org/aaemap by clicking on the right-most of the four icons on the bottom-left. Put that file it in the same directory or folder as this notebook and load the data using the code below

In [ ]:
def reshape_jsonarrays(item):
    time = np.array([d[0] for d in item[1]], dtype='datetime64')
    lat = np.array([d[1] for d in item[1]], dtype='float')
    lon = np.array([d[2] for d in item[1]], dtype='float')
    return {'name':item[0], 'time':time, 'lat':lat, 'lon':lon}

with open("aaedrifters.json") as fp:
    jsondata = json.load(fp)

drifters = [reshape_jsonarrays(item) for item in jsondata.items()]

Have a quick look of the content of the new `drifter` list by running the following cell:

In [ ]:
drifters

The drifter pairs were deployed on a straight line along the cruise track between approximately (56.0°S, 157.2°E) and (58.8°S, 153.5°E) over a 25 h period starting on 11 December 2013.  

Make a simple plot of all trajectories in `drifters` using `plt.plot`, both for the complete duration of the experiment and the first ten days after release.

To get a better view on the trajectories, it might be nice to show the continents as well. For this we use the `cartopy` package. Now run the following cell and see the resulting map.

In [ ]:
# We use the Platecarree projection centered at 180 degrees longitude
# on Cartopy's website you can find other projections to play around with
projection = cartopy.crs.PlateCarree(central_longitude=180)

# whenever adding data having latitude-longitude coordinates, use PlateCarree() as coordinate reference system
data_crs = cartopy.crs.PlateCarree()

# initialisation of the figure
fig = plt.figure(figsize=(12, 5))
ax = fig.add_subplot(1, 1, 1, projection=projection)

# plot coastlines and colour the land surfaces
ax.coastlines(resolution='50m')
ax.add_feature(cartopy.feature.LAND)

# plot trajectories
for d in drifters[:]:
    ax.plot(d['lon'], d['lat'], transform=data_crs, lw=0.8)

# plot grid lines and format the labels
# xlocs is needed because the locator does not work so well when crossing 180 degrees longitude
gl = ax.gridlines(crs=data_crs, draw_labels=['bottom','left'], linewidth=0.5,
              color='gray', alpha=0.5, linestyle='--', xlocs=range(-180,181,10))
gl.xformatter = cartopy.mpl.gridliner.LONGITUDE_FORMATTER
gl.yformatter = cartopy.mpl.gridliner.LATITUDE_FORMATTER

# zoom out to see more land masses
ax.set_extent((142,228,-77,-32), crs=data_crs)

Complete the function `drifter_velocity` below that computes the zonal and meridional velocities of a drifter. The function takes an element of `drifters` as an argument and returns two arrays, `u` and `v`. The remaining code will plot your results for drifter 130263. Make sure the vectors are aligned with the trajectory.

In [ ]:
# ANSWER
def drifter_velocity(data):
    """compute zonal and meridional velocities of drifter
    data : dictionary containing arrays time, lat and lon
    u, v : array, zonal (meridional) velocity
    """
    # enter your code here
    return u,v

data = drifters[4]
u,v = drifter_velocity(data)

crs = cartopy.crs.PlateCarree()
ax = plt.axes(projection=crs)
pids = slice(1200,1550,4)
Q = ax.quiver(data['lon'][pids], data['lat'][pids], u[pids], v[pids], transform=crs, lw=0.5)
ax.quiverkey(Q, 0.92, 0.05, 0.5, label='0.5 m/s')
ax.set_title(f"velocity of drifter {data['name']}")
ax.gridlines(draw_labels=['left','bottom'], dms=True)
plt.show()

Now use the function `drifter_velocity` to create maps of zonal and meridional velocity for all drifters. Part of the code to plot the results has been given below.  
Hint: use `ax.scatter` to change colour along the trajectories.

In [ ]:
# compute velocities here

In [ ]:
ax = plt.axes(projection=cartopy.crs.PlateCarree(central_longitude=180))

# plot results for u here

ax.set_title("zonal velocity")
ax.coastlines('50m')
ax.gridlines(draw_labels=['left','bottom'], transform=cartopy.crs.PlateCarree(), xlocs=range(-180,180,10))
ax.figure.colorbar(p, orientation='horizontal', extend='both', label="u (m/s)", aspect=50, pad=0.08)

In [ ]:
ax = plt.axes(projection=cartopy.crs.PlateCarree(central_longitude=180))

# plot results for v here

ax.set_title("meridional velocity")
ax.coastlines('50m')
ax.gridlines(draw_labels=['left','bottom'], transform=cartopy.crs.PlateCarree(), xlocs=range(-180,180,10))
ax.figure.colorbar(p, orientation='horizontal', extend='both', label="v (m/s)", aspect=50, pad=0.08)

The drifters have been released in pairs, about 13 meter apart on either side of the ship. This makes it possible to analyze the separation distance, which is a measure for dispersion. The cell below creates a list, `dists`, of dictionaries containing time, launching latitude, names and distance, $D$, between the drifters of each pair.  
Run it now and plot the drifter separation as a function of time for all drifters for the first ten days after launch. You may change the scale of the y-axis to logarithmic or symmetric logarithmic with `ax.set_yscale`. How does the separation after ten days depend on launching latitude?

In [ ]:
def separation(data1, data2):
    """calculate separation distance between two drifters"""
    time1 = data1['time']
    time2 = data2['time']
    time = np.array(sorted(set(time1).intersection(set(time2))))
    tids1 = np.searchsorted(time1, time)
    tids2 = np.searchsorted(time2, time)
    lats1 = data1['lat'][tids1]
    lons1 = data1['lon'][tids1]
    lats2 = data2['lat'][tids2]
    lons2 = data2['lon'][tids2]
    dist = [geopy.distance.distance((lat1,lon1),(lat2,lon2)).m for (lat1, lon1, lat2, lon2)
            in zip(lats1, lons1, lats2, lons2)]
    return {'name1':data1['name'], 'name2':data2['name'], 'time':time,
            'lat0':data1['lat'][0], 'distance':np.array(dist)}

lats = np.array([d['lat'][0] for d in drifters])
pairids = lats.argsort().reshape((-1,2)).tolist()
dists = []
for n,(i,j) in enumerate(pairids):
    print(f"\ranalyzing pair {n+1}/{len(pairids)}", end="")
    dists.append(separation(drifters[i], drifters[j]))

It is time to compare your results. Create a plot of the pairwise dispersion, $D^2$, similar to figure 5a in the [article](https://doi.org/10.1002/2015JC010972). You do not need to fit the data and calculate the slopes, but do check if the impact of the outlier drifter pair on the mean is similar.